In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-10-08 07:54:07--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  32.9MB/s    in 0.8s    

2024-10-08 07:54:08 (32.9 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# add your code here - consider creating a new cell for each section of code

user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]
df_ratings = df_ratings[df_ratings['isbn'].isin(book_counts[book_counts >= 100].index)]

In [25]:
# Create a user-item matrix
user_item_matrix = df_ratings.pivot(index='user', columns='isbn', values='rating').fillna(0)
user_item_sparse = csr_matrix(user_item_matrix.values)

# Fit KNN model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(user_item_sparse.T)

def get_recommends(book=""):
    try:
        # Find the index of the book
        book_index = df_books[df_books['title'] == book].index[0]
        book_isbn = df_books.iloc[book_index]['isbn']
        book_id = user_item_matrix.columns.get_loc(book_isbn)

        # Find similar books
        distances, indices = model_knn.kneighbors(user_item_sparse.T[book_id], n_neighbors=6)
        recommended_books = []

        for i in range(1, len(distances.flatten())):
            similar_isbn = user_item_matrix.columns[indices.flatten()[i]]
            similar_title = df_books[df_books['isbn'] == similar_isbn]['title'].values[0]
            recommended_books.append([similar_title, distances.flatten()[i]])

        return [book, recommended_books]
    except IndexError:
        return f"Book '{book}' not found in the dataset."

In [29]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  print("recommends[0]: " + str(recommends[0]))
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    print("You haven't returned the correct book recommends[0]")
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
      print("You have returned an extra book, recommends[1]["+str(i)+"][0]: " + str(recommends[1][i][0]))
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      print("You have not returned the correct distance, recommends[1]["+str(i)+"][1]: " + str(recommends[1][i][1]) + " should be " + str(recommended_books_dist[i]))
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.7234864], ['I Know This Much Is True', 0.7677075], ['The Surgeon', 0.7699411], ['The Weight of Water', 0.77085835], ["I'll Be Seeing You", 0.8016211]]]
recommends[0]: Where the Heart Is (Oprah's Book Club (Paperback))
You have returned an extra book, recommends[1][0][0]: The Lovely Bones: A Novel
You have not returned the correct distance, recommends[1][0][1]: 0.7234864 should be 0.8
You haven't passed yet. Keep trying!
